<a href="https://colab.research.google.com/github/EpicMahmoudSammourRTA/Future_Scanner/blob/main/Future_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##install required packages

In [52]:
!pip install PyMuPDF
!pip install fasttext

##Import all packages

In [53]:
import pandas as pd
import os
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import fitz
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import fasttext
import csv
import random
from os.path import join, dirname

##Use the NLTK Downloader to obtain the resource

In [54]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#To create the needed directories
### please upload some pdf files in the directories 

In [55]:
!mkdir /Output
!mkdir /Files1
!mkdir /Files2

mkdir: cannot create directory ‘/Output’: File exists
mkdir: cannot create directory ‘/Files1’: File exists
mkdir: cannot create directory ‘/Files2’: File exists


##Set the following paths

In [56]:

Output_Path = '/Output'  # Output data path
File1_Path = '/Files1' # Output data path
File2_Path = '/Files2'  # Output data path


# Load File set 1
##Place all the files from File1 folder into File1_Path
##pdf_reader function reads all the documents/articles in pdf format and combine in one dataframe.

In [57]:
os.chdir(File1_Path)

files = os.listdir(File1_Path)

Key_Trend = []
Trend = []
File_Name = []

for file in files:
    Key_Trend.append(file.split('_')[0].replace('.pdf', ''))
    Trend.append(file.split('_')[-1][:-4])
    File_Name.append(file)

file1 = pd.DataFrame(zip(Key_Trend, Trend, File_Name), columns=["Label", "Sub_Trend", "File_Name"])


def pdf_reader(rows):
    page_text = ''
    doc = fitz.open(rows)
    for i in range(doc.pageCount):
        page = doc.loadPage(i)
        page_str = page.getText("text")
        page_text = page_text + page_str
    return page_text


file1["Raw_Data"] = file1["File_Name"].apply(pdf_reader)

file1 = file1.loc[:, ["Label", "File_Name", "Raw_Data"]]
file1

,Label,File_Name,Raw_Data
0,3D Mobility,3D Mobility.pdf,3D Mobility\nModified at:\n23.05.2020\nModifie...
1,3D Mobility,3D Mobility_Autonomous Air Vehicles.pdf,Autonomous Air Vehicles\nModified at:\n16.06.2...
2,3D Mobility,3D Mobility_Ambulance Drones - Remote Healthca...,Ambulance Drones - Remote Healthcare 2.0\nModi...
3,3D Mobility,3D Mobility_Autonomous Drone.pdf,Autonomous Drone\nModified at:\n13.06.2019\nMo...


#Load File set 2
##Place all the files from File2 folder into File2_Path
##pdf_reader function reads all the documents/articles in pdf format and combine in one dataframe.

In [58]:
os.chdir(File2_Path)

files = os.listdir(File2_Path)

Key_Trend1 = []
Key_Trend2 = []
Key_Trend3 = []
Key_Trend4 = []
File_Name = []

for file in files:
    file_list = file.split('__')
    if len(file_list) == 2:
        Key_Trend1.append(file_list[0])
        Key_Trend2.append('')
        Key_Trend3.append('')
        Key_Trend4.append('')
        File_Name.append(file)
    elif len(file_list) == 3:
        Key_Trend1.append(file_list[0])
        Key_Trend2.append(file_list[1])
        Key_Trend3.append('')
        Key_Trend4.append('')
        File_Name.append(file)
    elif len(file_list) == 4:
        Key_Trend1.append(file_list[0])
        Key_Trend2.append(file_list[1])
        Key_Trend3.append(file_list[2])
        Key_Trend4.append('')
        File_Name.append(file)
    elif len(file_list) == 5:
        Key_Trend1.append(file_list[0])
        Key_Trend2.append(file_list[1])
        Key_Trend3.append(file_list[2])
        Key_Trend4.append(file_list[3])
        File_Name.append(file)

file2 = pd.DataFrame(zip(Key_Trend1, Key_Trend2, Key_Trend3, Key_Trend4, File_Name),
                     columns=["Label1", "Label2", "Label3", "Label4", "File_Name"])

df1 = file2.loc[:, ["Label1", "File_Name"]]

df1.rename(columns={"Label1": "Label"}, inplace=True)

df2 = file2.loc[file2["Label2"] != '', ["Label2", "File_Name"]]

df2.rename(columns={"Label2": "Label"}, inplace=True)

df3 = file2.loc[file2["Label3"] != '', ["Label3", "File_Name"]]

df3.rename(columns={"Label3": "Label"}, inplace=True)

df4 = file2.loc[file2["Label4"] != '', ["Label4", "File_Name"]]

df4.rename(columns={"Label4": "Label"}, inplace=True)

file2 = df1.append([df2, df3, df4], ignore_index=True)


def pdf_reader(rows):
    page_text = ''
    doc = fitz.open(rows)
    for i in range(doc.pageCount):
        page = doc.loadPage(i)
        page_str = page.getText("text")
        page_text = page_text + page_str
    return page_text


file2["Raw_Data"] = file2["File_Name"].apply(pdf_reader)

base_data = file1.append([file2], ignore_index=True)
base_data

,Label,File_Name,Raw_Data
0,3D Mobility,3D Mobility.pdf,3D Mobility\nModified at:\n23.05.2020\nModifie...
1,3D Mobility,3D Mobility_Autonomous Air Vehicles.pdf,Autonomous Air Vehicles\nModified at:\n16.06.2...
2,3D Mobility,3D Mobility_Ambulance Drones - Remote Healthca...,Ambulance Drones - Remote Healthcare 2.0\nModi...
3,3D Mobility,3D Mobility_Autonomous Drone.pdf,Autonomous Drone\nModified at:\n13.06.2019\nMo...
4,AI and Advanced Machine Learning,AI and Advanced Machine Learning__Self-Driving...,The technology that could end \ntraffic jams \...
5,Alternative Energy Sources,Alternative Energy Sources__Local Life and Gre...,A new high-power fuel cell operates at double ...
6,Alternative Energy Sources,Alternative Energy Sources__Local Life and Gre...,Are electric vehicles really so climate \nfrie...
7,AI and Advanced Machine Learning,AI and Advanced Machine Learning__The Death of...,The Era of Big Data is coming to an end as the...
8,AI and Advanced Machine Learning,AI and Advanced Machine Learning__How to Set U...,How to Set Up an AI Center of Excellence \nArt...
9,Alternative Energy Sources,Alternative Energy Sources__Converting Water a...,Converting Water and CO2 into Fuel \nResearche...


#Oversampling to train the model
#Oversampling is done for imbalance class

In [59]:
max_length = 60


def oversampling(rows):
    global max_length
    df = base_data.loc[base_data["Label"] == rows["Label"], :]
    ratio = len(df) / max_length
    if ratio <= 0.40:
        token_text = word_tokenize(rows["Raw_Data"])
        random.shuffle(token_text)
        text = (" ").join(token_text)
        return text
    else:
        return ""


wn = nltk.WordNetLemmatizer()

base_data["Oversample_Text"] = base_data[["Label", "Raw_Data"]].apply(oversampling, axis=1)

os_df = base_data.loc[base_data["Oversample_Text"] != "", ["Label", "File_Name", "Oversample_Text"]]

os_df.rename(columns={"Oversample_Text": "Raw_Data"}, inplace=True)

base_data.drop(["Oversample_Text"], inplace=True, axis=1)

base_data = base_data.append([os_df], ignore_index=True)

base_data_v1 = base_data.loc[~(base_data["Label"].isin(
    ["Novel Payment Systems", "Biometrics and Human-Machine Interface", "n_Self-Driving Transport"])), :]

base_data_v1["Label"] = base_data_v1["Label"].apply(lambda x: x.replace(' ', ''))

base_data_v1['Label'] = ['__label__' + s for s in base_data_v1['Label']]

base_data_v1 = base_data_v1.groupby(['Raw_Data'])['Label'].apply(' '.join).reset_index()

file_name = base_data.drop_duplicates(subset=['Raw_Data'])

base_data_v2 = base_data_v1.merge(file_name[['Raw_Data', "File_Name"]], left_on='Raw_Data', right_on='Raw_Data',
                                  how='left')
base_data_v2

,Raw_Data,Label,File_Name
0,. and an vertical area 2.0 has the Flying that...,__label__3DMobility,3D Mobility.pdf
1,. with much a optimized novel enabled this of ...,__label__LocalLifeandGreenEconomy,Alternative Energy Sources__Local Life and Gre...
2,3D Mobility\nModified at:\n23.05.2020\nModifie...,__label__3DMobility,3D Mobility.pdf
3,A new high-power fuel cell operates at double ...,__label__AlternativeEnergySources __label__Loc...,Alternative Energy Sources__Local Life and Gre...
4,Ambulance Drones - Remote Healthcare 2.0\nModi...,__label__3DMobility,3D Mobility_Ambulance Drones - Remote Healthca...
5,Are electric vehicles really so climate \nfrie...,__label__AlternativeEnergySources __label__Loc...,Alternative Energy Sources__Local Life and Gre...
6,Autonomous Air Vehicles\nModified at:\n16.06.2...,__label__3DMobility,3D Mobility_Autonomous Air Vehicles.pdf
7,Autonomous Drone\nModified at:\n13.06.2019\nMo...,__label__3DMobility,3D Mobility_Autonomous Drone.pdf
8,Converting Water and CO2 into Fuel \nResearche...,__label__AlternativeEnergySources,Alternative Energy Sources__Converting Water a...
9,How to Set Up an AI Center of Excellence \nArt...,__label__AIandAdvancedMachineLearning,AI and Advanced Machine Learning__How to Set U...


#Finding Document Features
#wc function is used to extract document features based on n-gram model and TF scores.

In [60]:
def wc(text, tfidf_vectorizer):
    doc_text = [text]
    tf_model = tfidf_vectorizer.fit_transform(doc_text)
    text_scored = tfidf_vectorizer.transform(doc_text)
    terms = tfidf_vectorizer.get_feature_names()
    scores = text_scored.toarray().flatten().tolist()
    data = list(zip(terms, scores))
    sorted_data = sorted(data, key=lambda x: x[1], reverse=True)
    top_words = sorted_data[:2]
    final_words = []
    for i in range(len(top_words)):
        final_words.append(top_words[i][0])
    return final_words

stop_words = stopwords.words('english')
newStopWords = ['created', 'modified', 'scout', 'strat', 'alun', 'rhydderch', 'description', 'tags', 'trends',
                'projects', 'page', 'steep', ]
stop_words.extend(newStopWords)
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

#Data Preprocessing
#preprocessing function is used to cleansing the Raw data and will generate Clean_Text and Document_Features

In [61]:
def preprocessing(rows):
    text = rows.lower()
    # /*********************Remove number*******************/
    text = re.sub(r'\d+', ' ', text)
    # /*****************Remove Punctuation****************/
    text = re.sub(r'[^\w\s]', ' ', text)
    # /*****************Remove \xa0****************/
    text = re.sub(r'\xa0', '', text)
    # /*****************Remove \x0c****************/
    text = re.sub(r'\x0c', '', text)
    #    /*****************Remove stop words************/
    token_text = word_tokenize(text)
    tokens_without_sw = [word for word in token_text if not word in stop_words]
    ##    text_lem = [wn.lemmatize(word) for word in tokens_without_sw]
    text_stem = [ps.stem(word) for word in tokens_without_sw]
    text = (" ").join(text_stem)
    # /***************Remove space line character*********/
    text = text.replace('\n', ' ')
    #    /********************Remove duplicate space**********/
    text = " ".join(text.split())
    # /********************************For word cloud**********************/
    text_lem = [wn.lemmatize(word) for word in tokens_without_sw]
    word_text = (" ").join(text_lem)
    # /***************Remove space line character*********/
    word_text = word_text.replace('\n', ' ')
    #    /********************Remove duplicate space**********/
    word_text = " ".join(word_text.split())
    tfidf_vectorizer = TfidfVectorizer()
    top_unigram_words = wc(word_text, tfidf_vectorizer)
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 2))
    top_bigram_words = wc(word_text, tfidf_vectorizer)
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(3, 3))
    top_trigram_words = wc(word_text, tfidf_vectorizer)
    combined_gram = top_unigram_words + top_bigram_words + top_trigram_words
    return text, combined_gram


base_data_v2["Clean_Text"], base_data_v2["Document_Features"] = zip(*base_data_v2["Raw_Data"].apply(preprocessing))
base_data_v2

,Raw_Data,Label,File_Name,Clean_Text,Document_Features
0,. and an vertical area 2.0 has the Flying that...,__label__3DMobility,3D Mobility.pdf,vertic area fli need lane skyscrap effect hydr...,"(mobility, drone, air ambulance, air land, air..."
1,. with much a optimized novel enabled this of ...,__label__LocalLifeandGreenEconomy,Alternative Energy Sources__Local Life and Gre...,much optim novel enabl electrod drone support ...,"(cell, fuel, gradient enabled, ph gradient, ph..."
2,3D Mobility\nModified at:\n23.05.2020\nModifie...,__label__3DMobility,3D Mobility.pdf,mobil life megac long sinc move three dimensio...,"(drone, mobility, air vehicle, autonomous dron..."
3,A new high-power fuel cell operates at double ...,__label__AlternativeEnergySources __label__Loc...,Alternative Energy Sources__Local Life and Gre...,new high power fuel cell oper doubl voltag tod...,"(cell, fuel, fuel cell, acid alkali, universit..."
4,Ambulance Drones - Remote Healthcare 2.0\nModi...,__label__3DMobility,3D Mobility_Ambulance Drones - Remote Healthca...,ambul drone remot healthcar ambul drone integr...,"(drone, ambulance, ambulance drone, drone remo..."
5,Are electric vehicles really so climate \nfrie...,__label__AlternativeEnergySources __label__Loc...,Alternative Energy Sources__Local Life and Gre...,electr vehicl realli climat friendli germani a...,"(emission, vehicle, electric vehicle, energy m..."
6,Autonomous Air Vehicles\nModified at:\n16.06.2...,__label__3DMobility,3D Mobility_Autonomous Air Vehicles.pdf,autonom air vehicl air born vehicl privat aero...,"(air, autonomous, air vehicle, autonomous air,..."
7,Autonomous Drone\nModified at:\n13.06.2019\nMo...,__label__3DMobility,3D Mobility_Autonomous Drone.pdf,autonom drone autonom drone uav multipl usag i...,"(drone, autonomous, autonomous drone, cargo de..."
8,Converting Water and CO2 into Fuel \nResearche...,__label__AlternativeEnergySources,Alternative Energy Sources__Converting Water a...,convert water co fuel research tri duplic natu...,"(carbon, process, carbon dioxide, carbon monox..."
9,How to Set Up an AI Center of Excellence \nArt...,__label__AIandAdvancedMachineLearning,AI and Advanced Machine Learning__How to Set U...,set ai center excel artifici intellig one powe...,"(ai, business, ai center, use case, new busine..."


#Model Building
#We are using Fasttext to build our mode

In [62]:
train_data = base_data_v2.loc[:, ["Label", "Clean_Text"]]

train_data.to_csv(Output_Path + '\\train_data.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE,
                  quotechar="", escapechar=" ")

model = fasttext.train_supervised(input=Output_Path + '\\train_data.txt', lr=0.5, epoch=100, loss='hs')

#Model Predictions


In [63]:
def predictions(rows):
    predicted_label_1 = model.predict(rows, k=-1)[0][0]
    predicted_label_1_probab = model.predict(rows, k=-1)[1][0]
    predicted_label_2 = model.predict(rows, k=-1)[0][1]
    predicted_label_2_probab = model.predict(rows, k=-1)[1][1]
    try:
        predicted_label_3 = model.predict(rows, k=-1)[0][2]
        predicted_label_3_probab = model.predict(rows, k=-1)[1][2]
    except:
        predicted_label_3 = ''
        predicted_label_3_probab = ''
    if predicted_label_1_probab >= 0.80 or predicted_label_2_probab < 0.15:
        return predicted_label_1, predicted_label_1_probab, ' ', ' ', ' ', ' '
    elif (predicted_label_1_probab + predicted_label_2_probab) >= 0.70:
        return predicted_label_1, predicted_label_1_probab, predicted_label_2, predicted_label_2_probab, ' ', ' '
    else:
        return predicted_label_1, predicted_label_1_probab, predicted_label_2, predicted_label_2_probab, predicted_label_3, predicted_label_3_probab


base_data_v2["predicted_label_1"], base_data_v2["predicted_label_1_probab"], base_data_v2["predicted_label_2"], \
base_data_v2["predicted_label_2_probab"], base_data_v2["predicted_label_3"], base_data_v2[
    "predicted_label_3_probab"] = zip(*base_data_v2["Clean_Text"].apply(predictions))
base_data_v2

,Raw_Data,Label,File_Name,Clean_Text,Document_Features,predicted_label_1,predicted_label_1_probab,predicted_label_2,predicted_label_2_probab,predicted_label_3,predicted_label_3_probab
0,. and an vertical area 2.0 has the Flying that...,__label__3DMobility,3D Mobility.pdf,vertic area fli need lane skyscrap effect hydr...,"(mobility, drone, air ambulance, air land, air...",__label__3DMobility,0.983097,,,,
1,. with much a optimized novel enabled this of ...,__label__LocalLifeandGreenEconomy,Alternative Energy Sources__Local Life and Gre...,much optim novel enabl electrod drone support ...,"(cell, fuel, gradient enabled, ph gradient, ph...",__label__AlternativeEnergySources,0.372457,__label__3DMobility,0.273557,__label__LocalLifeandGreenEconomy,0.18699
2,3D Mobility\nModified at:\n23.05.2020\nModifie...,__label__3DMobility,3D Mobility.pdf,mobil life megac long sinc move three dimensio...,"(drone, mobility, air vehicle, autonomous dron...",__label__3DMobility,0.981759,,,,
3,A new high-power fuel cell operates at double ...,__label__AlternativeEnergySources __label__Loc...,Alternative Energy Sources__Local Life and Gre...,new high power fuel cell oper doubl voltag tod...,"(cell, fuel, fuel cell, acid alkali, universit...",__label__AlternativeEnergySources,0.372457,__label__3DMobility,0.273557,__label__LocalLifeandGreenEconomy,0.18699
4,Ambulance Drones - Remote Healthcare 2.0\nModi...,__label__3DMobility,3D Mobility_Ambulance Drones - Remote Healthca...,ambul drone remot healthcar ambul drone integr...,"(drone, ambulance, ambulance drone, drone remo...",__label__3DMobility,0.998306,,,,
5,Are electric vehicles really so climate \nfrie...,__label__AlternativeEnergySources __label__Loc...,Alternative Energy Sources__Local Life and Gre...,electr vehicl realli climat friendli germani a...,"(emission, vehicle, electric vehicle, energy m...",__label__AlternativeEnergySources,0.652330,__label__LocalLifeandGreenEconomy,0.22581,,
6,Autonomous Air Vehicles\nModified at:\n16.06.2...,__label__3DMobility,3D Mobility_Autonomous Air Vehicles.pdf,autonom air vehicl air born vehicl privat aero...,"(air, autonomous, air vehicle, autonomous air,...",__label__3DMobility,0.979486,,,,
7,Autonomous Drone\nModified at:\n13.06.2019\nMo...,__label__3DMobility,3D Mobility_Autonomous Drone.pdf,autonom drone autonom drone uav multipl usag i...,"(drone, autonomous, autonomous drone, cargo de...",__label__3DMobility,0.984569,,,,
8,Converting Water and CO2 into Fuel \nResearche...,__label__AlternativeEnergySources,Alternative Energy Sources__Converting Water a...,convert water co fuel research tri duplic natu...,"(carbon, process, carbon dioxide, carbon monox...",__label__AlternativeEnergySources,0.647796,__label__LocalLifeandGreenEconomy,0.22498,,
9,How to Set Up an AI Center of Excellence \nArt...,__label__AIandAdvancedMachineLearning,AI and Advanced Machine Learning__How to Set U...,set ai center excel artifici intellig one powe...,"(ai, business, ai center, use case, new busine...",__label__AIandAdvancedMachineLearning,0.803985,,,,


#This function is used to clean the model output

In [64]:
def clean(rows):
    text = rows.replace("__label__", '')
    return text


base_data_v2["Label"] = base_data_v2["Label"].apply(clean)
base_data_v2["predicted_label_1"] = base_data_v2["predicted_label_1"].apply(clean)
base_data_v2["predicted_label_2"] = base_data_v2["predicted_label_2"].apply(clean)
base_data_v2["predicted_label_3"] = base_data_v2["predicted_label_3"].apply(clean)

base_data_v2.rename(columns={"Label": "Actual_Label"}, inplace=True)

base_data_v3 = base_data_v2.loc[:,
               ["File_Name", "Actual_Label", "Document_Features", "predicted_label_1", "predicted_label_1_probab",
                "predicted_label_2", "predicted_label_2_probab", "predicted_label_3", "predicted_label_3_probab"]]
base_data_v3

,File_Name,Actual_Label,Document_Features,predicted_label_1,predicted_label_1_probab,predicted_label_2,predicted_label_2_probab,predicted_label_3,predicted_label_3_probab
0,3D Mobility.pdf,3DMobility,"(mobility, drone, air ambulance, air land, air...",3DMobility,0.983097,,,,
1,Alternative Energy Sources__Local Life and Gre...,LocalLifeandGreenEconomy,"(cell, fuel, gradient enabled, ph gradient, ph...",AlternativeEnergySources,0.372457,3DMobility,0.273557,LocalLifeandGreenEconomy,0.18699
2,3D Mobility.pdf,3DMobility,"(drone, mobility, air vehicle, autonomous dron...",3DMobility,0.981759,,,,
3,Alternative Energy Sources__Local Life and Gre...,AlternativeEnergySources LocalLifeandGreenEcon...,"(cell, fuel, fuel cell, acid alkali, universit...",AlternativeEnergySources,0.372457,3DMobility,0.273557,LocalLifeandGreenEconomy,0.18699
4,3D Mobility_Ambulance Drones - Remote Healthca...,3DMobility,"(drone, ambulance, ambulance drone, drone remo...",3DMobility,0.998306,,,,
5,Alternative Energy Sources__Local Life and Gre...,AlternativeEnergySources LocalLifeandGreenEconomy,"(emission, vehicle, electric vehicle, energy m...",AlternativeEnergySources,0.652330,LocalLifeandGreenEconomy,0.22581,,
6,3D Mobility_Autonomous Air Vehicles.pdf,3DMobility,"(air, autonomous, air vehicle, autonomous air,...",3DMobility,0.979486,,,,
7,3D Mobility_Autonomous Drone.pdf,3DMobility,"(drone, autonomous, autonomous drone, cargo de...",3DMobility,0.984569,,,,
8,Alternative Energy Sources__Converting Water a...,AlternativeEnergySources,"(carbon, process, carbon dioxide, carbon monox...",AlternativeEnergySources,0.647796,LocalLifeandGreenEconomy,0.22498,,
9,AI and Advanced Machine Learning__How to Set U...,AIandAdvancedMachineLearning,"(ai, business, ai center, use case, new busine...",AIandAdvancedMachineLearning,0.803985,,,,


#Export the output results

In [65]:
base_data_v3.to_excel(Output_Path + '\\Scoring_Data.xlsx')